In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

# Kiểm tra index
if not es.indices.exists(index="datatest1"):
    raise Exception("Index không tồn tại")

# Thêm trường embedding
mapping_update = {
    "properties": {
        "embedding": {
            "type": "dense_vector",
            "dims": 384,  # Số chiều của model (MiniLM=384)
            "index": True if es.info()["version"]["number"].startswith("8") else False,
        }
    }
}

es.indices.put_mapping(index="datatest1", body=mapping_update)

In [ ]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Lấy tất cả documents
docs = []
scroll = es.search(
    index="datatest1", 
    scroll="2m", 
    size=1000,
    body={"query": {"match_all": {}}, "_source": ["title"]} 
)

while len(scroll["hits"]["hits"]) > 0:
    docs.extend(scroll["hits"]["hits"])
    scroll = es.scroll(scroll_id=scroll["_scroll_id"], scroll="2m")

batch_size = 100
for i in tqdm(range(0, len(docs), batch_size)):
    batch = docs[i:i+batch_size]
    
    # Tạo embedding hàng loạt
    texts = [doc["_source"]["title"] for doc in batch]
    embeddings = model.encode(texts, batch_size=batch_size)
    
    bulk_actions = []
    for doc, emb in zip(batch, embeddings):
        bulk_actions.append({
            "update": {
                "_index": "datatest1",
                "_id": doc["_id"]
            }
        })
        bulk_actions.append({
            "doc": {
                "embedding": emb.tolist()
            }
        })
    
    es.bulk(body=bulk_actions)

In [ ]:
# Lấy 1 document
sample = es.search(index="datatest1", size=1)["hits"]["hits"][0]
print("Document mẫu:", sample["_source"].keys())  

In [97]:
query_text = "thiếu nhi"
query_vector = model.encode(query_text).tolist()  

response = es.search(
    index="datatest1",
    body={
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {"query_vector": query_vector}
                }
            }
        },
        "size": 10 
    }
)

#  In kết quả 
if response["hits"]["hits"]:
    print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}':(semantic search)\n")
    for i, hit in enumerate(response["hits"]["hits"], 1):
        print(f"#{i} | Score: {hit['_score']:.4f}")
        print(f"   Title: {hit['_source'].get('title', 'N/A')}")
        print(f"   ID: {hit['_id']}")
        print("-" * 50)
else:
    print("Không tìm thấy kết quả phù hợp")

Tìm thấy 10 kết quả cho 'thiếu nhi':(semantic search)

#1 | Score: 1.7254
   Title: Chuyện lì xì cho trẻ ngày tết
   ID: BMrD-ZUBTAfSoAzdR4IS
--------------------------------------------------
#2 | Score: 1.6608
   Title: Người trẻ giấy dó
   ID: ZMrD-ZUBTAfSoAzdR4IS
--------------------------------------------------
#3 | Score: 1.6344
   Title: Thiên lương
   ID: RMrD-ZUBTAfSoAzdR4IS
--------------------------------------------------
#4 | Score: 1.6222
   Title: Thầy và trò
   ID: 6MrD-ZUBTAfSoAzdR34Q
--------------------------------------------------
#5 | Score: 1.6209
   Title: Quà quê
   ID: dsrD-ZUBTAfSoAzdR4AR
--------------------------------------------------
#6 | Score: 1.6112
   Title: Đẻ nhiều con trai hơn con gái
   ID: gsrD-ZUBTAfSoAzdR4ER
--------------------------------------------------
#7 | Score: 1.6039
   Title: Tin vắn
   ID: 28rD-ZUBTAfSoAzdR34Q
--------------------------------------------------
#8 | Score: 1.6039
   Title: Tin vắn
   ID: wcrD-ZUBTAfSoAzdR4ES
------

C:\Users\ABC\AppData\Local\Temp\ipykernel_11408\953283086.py:4: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(


In [93]:
query_text ="thiếu nhi"

response = es.search(
    index="datatest1",
    body={
        "query": {
            "multi_match": { 
                "query": query_text,
                "fields": ["title"],  
                "fuzziness": "AUTO" 
            }
        },
        "size": 10  
    }
)
if response["hits"]["hits"]:
    print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}':(full-text search)\n")
    for i, hit in enumerate(response["hits"]["hits"], 1):
        print(f"#{i} | Score: {hit['_score']:.4f}")
        print(f"   Title: {hit['_source'].get('title', 'N/A')}")
        print(f"   ID: {hit['_id']}")
        print("-" * 50)
else:
    print("Không tìm thấy kết quả phù hợp")

Tìm thấy 10 kết quả cho 'thiếu nhi':(full-text search)

#1 | Score: 8.5968
   Title: Vận động sáng tác cho thiếu nhi năm 2011
   ID: wsrD-ZUBTAfSoAzdR4ES
--------------------------------------------------
#2 | Score: 8.5046
   Title: Vẻ đẹp của thiếu nữ người Việt gốc Phi dự thi hoa hậu
   ID: iMrD-ZUBTAfSoAzdR4ER
--------------------------------------------------
#3 | Score: 6.1520
   Title: Hằn lún đường chủ yếu do thiết kế, thi công
   ID: _MrD-ZUBTAfSoAzdR4ES
--------------------------------------------------
#4 | Score: 5.9317
   Title: Muôn kiểu thiết kế đảo bếp đẹp cho không gian nhỏ
   ID: DMrD-ZUBTAfSoAzdR4IS
--------------------------------------------------
#5 | Score: 5.7712
   Title: Nhà nổi trên cọc tre
   ID: 5crD-ZUBTAfSoAzdR34Q
--------------------------------------------------
#6 | Score: 5.7266
   Title: Nhiều thiết bị độc tại triển lãm Nghe nhìn Hà Nội 2014
   ID: VcrD-ZUBTAfSoAzdR4AR
--------------------------------------------------
#7 | Score: 5.7266
   Title: Ng

C:\Users\ABC\AppData\Local\Temp\ipykernel_11408\3680889384.py:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(


In [90]:
query_text = "thiếu nhi"
query_vector = model.encode(query_text).tolist()  

response = es.search(
    index="datatest1",
    body={
        "query": {
            "bool": {
                "should": [ 
                    {
                        "multi_match": {
                            "query": query_text,
                            "fields": ["title^3", "description^2", "tags^1"],  
                            "fuzziness": "AUTO"
                       }
                    },
                    {
                        "script_score": {
                            "query": {"match_all": {}},
                            "script": {
                                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                                "params": {"query_vector": query_vector}
                            }
                        }
                    }
                ],
                "minimum_should_match": 1
            }
        },
        "size": 10,
        "_source": ["title"] 
    }
)
# In kết quả
if response["hits"]["hits"]:
    print(f"Kết hợp Full-Text + Semantic Search cho '{query_text}':\n")
    for i, hit in enumerate(response["hits"]["hits"], 1):
        print(f"#{i} | Điểm tổng: {hit['_score']:.4f}")
        print(f"   Tiêu đề: {hit['_source'].get('title', 'N/A')}")
        print(f"   ID: {hit['_id']}")
        print("-" * 50)
else:
    print("Không tìm thấy kết quả phù hợp")

Kết hợp Full-Text + Semantic Search cho 'thiếu nhi':

#1 | Điểm tổng: 27.3858
   Tiêu đề: Vận động sáng tác cho thiếu nhi năm 2011
   ID: wsrD-ZUBTAfSoAzdR4ES
--------------------------------------------------
#2 | Điểm tổng: 26.8511
   Tiêu đề: Vẻ đẹp của thiếu nữ người Việt gốc Phi dự thi hoa hậu
   ID: iMrD-ZUBTAfSoAzdR4ER
--------------------------------------------------
#3 | Điểm tổng: 19.5277
   Tiêu đề: Hằn lún đường chủ yếu do thiết kế, thi công
   ID: _MrD-ZUBTAfSoAzdR4ES
--------------------------------------------------
#4 | Điểm tổng: 18.9420
   Tiêu đề: Muôn kiểu thiết kế đảo bếp đẹp cho không gian nhỏ
   ID: DMrD-ZUBTAfSoAzdR4IS
--------------------------------------------------
#5 | Điểm tổng: 18.5580
   Tiêu đề: Nhà nổi trên cọc tre
   ID: 5crD-ZUBTAfSoAzdR34Q
--------------------------------------------------
#6 | Điểm tổng: 18.5033
   Tiêu đề: Nghi Lộc: Trình diễn sử dụng thiết bị gieo, trỉa ngô, lạc
   ID: iMrD-ZUBTAfSoAzdR38Q
---------------------------------------

C:\Users\ABC\AppData\Local\Temp\ipykernel_11408\2730045775.py:4: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(
